<a href="https://colab.research.google.com/github/JayKarhade/Breath-Classification/blob/main/Breath_Classification_DNN_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pylab as plt
import pandas as pd
import sklearn
from keras.utils import to_categorical

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/breath_detect/dataset.csv')
df2 = pd.read_csv('/content/drive/MyDrive/breath_detect/class_labels.csv')
x = df1.to_numpy()[:,1:390]
y = df2.to_numpy()[:,1]
print(x.shape,y.shape)

#Shuffle data
indices = list(range(x.shape[0]))
np.random.shuffle(indices)
x = x[indices]
y=y[indices]
y=to_categorical(y)
#x = x.reshape(x.shape[0],x.shape[1],1)

##Train-Test split
from sklearn.model_selection import train_test_split
#y= to_categorical(y)
x_train_raw, x_test_raw, y_train_raw, y_test_raw = train_test_split(x,y, test_size=0.25, random_state=1)
x_train_raw, x_val_raw, y_train_raw, y_val_raw = train_test_split(x_train_raw,y_train_raw, test_size=2/8, random_state=1)

label_names = ['Normal', 'Slow', 'Fast', 'Deep']

(455, 389) (455,)


In [ ]:
import scipy.stats as st
from scipy.fftpack import fft, fftfreq 
from scipy.signal import argrelextrema
import operator

def stat_area_features(x, Te=1.0):

    mean_ts = np.mean(x, axis=1).reshape(-1,1) # mean
    max_ts = np.amax(x, axis=1).reshape(-1,1) # max
    min_ts = np.amin(x, axis=1).reshape(-1,1) # min
    std_ts = np.std(x, axis=1).reshape(-1,1) # std
    skew_ts = st.skew(x, axis=1).reshape(-1,1) # skew
    kurtosis_ts = st.kurtosis(x, axis=1).reshape(-1,1) # kurtosis 
    iqr_ts = st.iqr(x, axis=1).reshape(-1,1) # interquartile rante
    mad_ts = np.median(np.sort(abs(x - np.median(x, axis=1).reshape(-1,1)),
                               axis=1), axis=1).reshape(-1,1) # median absolute deviation
    area_ts = np.trapz(x, axis=1, dx=Te).reshape(-1,1) # area under curve
    sq_area_ts = np.trapz(x ** 2, axis=1, dx=Te).reshape(-1,1) # area under curve ** 2

    return np.concatenate((mean_ts,max_ts,min_ts,std_ts,skew_ts,kurtosis_ts,
                           iqr_ts,mad_ts,area_ts,sq_area_ts), axis=1)

def frequency_domain_features(x, Te=1.0):

    # As the DFT coefficients and their corresponding frequencies are symetrical arrays
    # with respect to the middle of the array we need to know if the number of readings 
    # in x is even or odd to then split the arrays...
    if x.shape[1]%2 == 0:
        N = int(x.shape[1]/2)
    else:
        N = int(x.shape[1]/2) - 1
    xf = np.repeat(fftfreq(x.shape[1],d=Te)[:N].reshape(1,-1), x.shape[0], axis=0) # frequencies
    dft = np.abs(fft(x, axis=1))[:,:N] # DFT coefficients   
    
    # statistical and area features
    dft_features = stat_area_features(dft, Te=1.0)
    # weighted mean frequency
    dft_weighted_mean_f = np.average(xf, axis=1, weights=dft).reshape(-1,1)
    # 5 first DFT coefficients 
    dft_first_coef = dft[:,:5]    
    # 5 first local maxima of DFT coefficients and their corresponding frequencies
    dft_max_coef = np.zeros((x.shape[0],5))
    dft_max_coef_f = np.zeros((x.shape[0],5))
    for row in range(x.shape[0]):
        # finds all local maximas indexes
        extrema_ind = argrelextrema(dft[row,:], np.greater, axis=0) 
        # makes a list of tuples (DFT_i, f_i) of all the local maxima
        # and keeps the 5 biggest...
        extrema_row = sorted([(dft[row,:][j],xf[row,j]) for j in extrema_ind[0]],
                             key=operator.itemgetter(0), reverse=True)[:5] 
        for i, ext in enumerate(extrema_row):
            dft_max_coef[row,i] = ext[0]
            dft_max_coef_f[row,i] = ext[1]    
    
    return np.concatenate((dft_features,dft_weighted_mean_f,dft_first_coef,
                           dft_max_coef,dft_max_coef_f), axis=1)

def make_feature_vector(x, Te=1.0):

    # Raw signals :  stat and area features
    features_xt = stat_area_features(x, Te=Te)
    
    # Jerk signals :  stat and area features
    features_xt_jerk = stat_area_features((x[:,1:]-x[:,:-1])/Te, Te=Te)
    
    # Raw signals : frequency domain features 
    features_xf = frequency_domain_features(x, Te=1/Te)
    
    # Jerk signals : frequency domain features 
    features_xf_jerk = frequency_domain_features((x[:,1:]-x[:,:-1])/Te, Te=1/Te)
        
    return np.concatenate((features_xt, features_xt_jerk, features_xf,features_xf_jerk), axis=1)

In [ ]:
X_train = make_feature_vector(x_train_raw, Te=1/50)
X_test = make_feature_vector(x_test_raw, Te=1/50)

print("X_train shape : {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))

X_train shape : (255, 72)
X_test shape: (114, 72)


In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
#model.add(Dense(500, input_dim=72, activation='relu'))
#model.add(Dense(250, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(4, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.fit(X_train, y_train_raw, epochs=10,validation_data=(X_test,y_test_raw), batch_size=10)


Epoch 1/10
26/26 [==============================] - 1s 9ms/step - loss: 0.9978 - accuracy: 0.5929 - val_loss: 0.6731 - val_accuracy: 0.7719
Epoch 2/10
26/26 [==============================] - 0s 3ms/step - loss: 0.4797 - accuracy: 0.8592 - val_loss: 0.4668 - val_accuracy: 0.8333
Epoch 3/10
26/26 [==============================] - 0s 3ms/step - loss: 0.3697 - accuracy: 0.8948 - val_loss: 0.4016 - val_accuracy: 0.8421
Epoch 4/10
26/26 [==============================] - 0s 3ms/step - loss: 0.2661 - accuracy: 0.9375 - val_loss: 0.3439 - val_accuracy: 0.8772
Epoch 5/10
26/26 [==============================] - 0s 3ms/step - loss: 0.1702 - accuracy: 0.9819 - val_loss: 0.2823 - val_accuracy: 0.9211
Epoch 6/10
26/26 [==============================] - 0s 3ms/step - loss: 0.1295 - accuracy: 0.9791 - val_loss: 0.2769 - val_accuracy: 0.8947
Epoch 7/10
26/26 [==============================] - 0s 3ms/step - loss: 0.1161 - accuracy: 0.9800 - val_loss: 0.2291 - val_accuracy: 0.9211
Epoch 8/10
26/26 [==

In [ ]:
Z=model.predict_classes(X_test)
y_test_class = np.zeros(y_test_raw.shape[0])
for i in range(y_test_raw.shape[0]):
  if y_test_raw[i][0]==1:
   y_test_class[i]=0
  elif y_test_raw[i][1]==1:
    y_test_class[i]=1
  elif y_test_raw[i][2]==1:
    y_test_class[i]=2
  elif y_test_raw[i][3]==1:
    y_test_class[i]=3

import sklearn
cm = sklearn.metrics.confusion_matrix(y_test_class,Z)
print(cm) 

[[22  3  2  0]
 [ 0 22  1  0]
 [ 0  0 41  0]
 [ 1  0  0 22]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
print(classification_report(y_test_class,Z, target_names=label_names))

              precision    recall  f1-score   support

      Normal       0.96      0.81      0.88        27
        Slow       0.88      0.96      0.92        23
        Fast       0.93      1.00      0.96        41
        Deep       1.00      0.96      0.98        23

    accuracy                           0.94       114
   macro avg       0.94      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114



In [ ]:
sklearn.metrics.precision_score(y_test_class,Z,average='micro')